# Text pre-processing 
Normalization and other things 

In [100]:
#!pip install pyspellchecker
#import nltk
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/allisonwun-
[nltk_data]     huikoh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [98]:
import itertools
import multiprocessing as mp
import numpy as np
import nltk
import pandas as pd
import re
import sklearn
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import nltk

from textblob import TextBlob

## Basic pre-processing before removing contractions

In [104]:
# load data 
train_df = pd.read_csv('/Users/allisonwun-huikoh/Google Drive/___hertie/2019-fall/e1291-ml/e1291-group-project/news-data/us_news_test.csv')
train_df = train_df[train_df['headline'].notnull()]
train_df = train_df[train_df['description'].notnull()]
train_df = train_df[train_df['text'].notnull()]
valid_df = pd.read_csv('/Users/allisonwun-huikoh/Google Drive/___hertie/2019-fall/e1291-ml/e1291-group-project/news-data/us_news_valid.csv')
valid_df.columns = [col.replace('validate.', '') for col in valid_df.columns]
valid_df = valid_df[valid_df['headline'].notnull()]
valid_df = valid_df[valid_df['description'].notnull()]
valid_df = valid_df[valid_df['text'].notnull()]

In [105]:
# inspecting data 
valid_df.dtypes

outlet           object
outlet_url       object
datetime         object
url_orig         object
headline         object
description      object
author           object
domain           object
topic_tags       object
text             object
section          object
news_keywords    object
subsection       object
paywall          object
provider         object
ideology         object
dtype: object

In [106]:
 # lowercase 
train_df['headline'] = train_df['headline'].apply(lambda x: " ".join(x.lower() for x in x.split()))
valid_df['headline'] = valid_df['headline'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train_df['description'] = train_df['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))
valid_df['description'] = valid_df['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train_df['text'] = train_df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
valid_df['text'] = valid_df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [107]:
# removing punctuation 
train_df['headline'] = train_df['headline'].str.replace('[^\w\s]','')
valid_df['headline'] = valid_df['headline'].str.replace('[^\w\s]','')
train_df['description'] = train_df['description'].str.replace('[^\w\s]','')
valid_df['description'] = valid_df['description'].str.replace('[^\w\s]','')
train_df['text'] = train_df['text'].str.replace('[^\w\s]','')
valid_df['text'] = valid_df['text'].str.replace('[^\w\s]','')

In [108]:
# removing stopwords 
stop = stopwords.words('english')
train_df['headline'] = train_df['headline'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
valid_df['headline'] = valid_df['headline'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train_df['description'] = train_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
valid_df['description'] = valid_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train_df['text'] = train_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
valid_df['text'] = valid_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [115]:
# common word removal 
freq1 = pd.Series(' '.join(train_df['headline']).split()).value_counts()[:10]
freq1 = list(freq1.index)
train_df['headline'] = train_df['headline'].apply(lambda x: " ".join(x for x in x.split() if x not in freq1))

freq2 = pd.Series(' '.join(valid_df['headline']).split()).value_counts()[:10]
freq2 = list(freq2.index)
valid_df['headline'] = valid_df['headline'].apply(lambda x: " ".join(x for x in x.split() if x not in freq2))

freq3 = pd.Series(' '.join(train_df['description']).split()).value_counts()[:10]
freq3 = list(freq3.index)
train_df['description'] = train_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq3))

freq4 = pd.Series(' '.join(valid_df['description']).split()).value_counts()[:10]
freq4 = list(freq4.index)
valid_df['description'] = valid_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq4))

freq5 = pd.Series(' '.join(train_df['text']).split()).value_counts()[:10]
freq5 = list(freq5.index)
train_df['text'] = train_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq5))

freq6 = pd.Series(' '.join(valid_df['text']).split()).value_counts()[:10]
freq6 = list(freq6.index)
valid_df['text'] = valid_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq6))


In [124]:
##### rare word removal
freq_1 = pd.Series(' '.join(train_df['headline']).split()).value_counts()[-10:]
freq_1 = list(freq_1.index)
train_df['headline'] = train_df['headline'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_1))

freq_2 = pd.Series(' '.join(valid_df['headline']).split()).value_counts()[-10:]
freq_2 = list(freq_2.index)
valid_df['headline'] = valid_df['headline'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_2))

freq_3 = pd.Series(' '.join(train_df['description']).split()).value_counts()[-10:]
freq_3 = list(freq_3.index)
train_df['description'] = train_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_3))

freq_4 = pd.Series(' '.join(valid_df['description']).split()).value_counts()[-10:]
freq_4 = list(freq_4.index)
valid_df['description'] = valid_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_4))

freq_5 = pd.Series(' '.join(train_df['text']).split()).value_counts()[-10:]
freq_5 = list(freq_5.index)
train_df['text'] = train_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_5))

freq_6 = pd.Series(' '.join(valid_df['text']).split()).value_counts()[-10:]
freq_6 = list(freq_6.index)
valid_df['text'] = valid_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_6))

In [ ]:
# spelling correction 
train_df['headline'].apply(lambda x: str(TextBlob(x).correct()))
valid_df['headline'].apply(lambda x: str(TextBlob(x).correct()))
train_df['description'].apply(lambda x: str(TextBlob(x).correct()))
valid_df['description'].apply(lambda x: str(TextBlob(x).correct()))
train_df['text'].apply(lambda x: str(TextBlob(x).correct()))
valid_df['text'].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
lemmatizer = WordNetLemmatizer()
tokenizer =  TweetTokenizer()
# spell = SpellChecker()

train_df.head()

In [92]:
def normalize_contractions(tokens):
	token_map = {
		"ain't" : 'is not',
		"aint" : 'is not',
		"aren't" : 'are not',
		"can't" : 'can not',
		"cannot" : 'can not',
		"could've" : 'could have',
		"couldn't" : 'could not',
		"couldnt" : 'could not',
		"didn't" : 'did not',
		"didnt" : 'did not',
		"doesn't" : 'does not',
		"doesnt" : 'does not',
		"don't" : 'do not',
		"gonna'" : 'going to',
		"gotta'" : 'got to',
		"hadn't" : 'had not',
		"hasn't" : 'has not',
		"haven't" : 'have not',
		"he'll" : 'he will',
		"he's" : 'he is',
		"he've" : 'he have',
		"how'd" : 'how did',
		"how'll" : 'how will',
		"how're" : 'how are',
		"how's" : 'how is',
		"i'd" : 'i would',
		"i'll" : 'i will',
		"i'm" : 'i am',
		"i'mm" : 'i am',
		"i've" : 'i have',
		"iäm" : 'i am',
		"isn't" : 'is not',
		"isnt" : 'is not',
		"it'd" : 'it would',
		"it'll" : 'it shall',
		"it's" : 'it is',
		"let's" : 'let us',
		"she'll" : 'she will',
		"she's" : 'she is',
		"should've" : 'should have',
		"shouldn't" : 'should not',
		"shouldnt" : 'should not',
		"that'll" : 'that will',
		"that's" : 'that is',
		"there's" : 'there is',
		"they'll" : 'they will',
		"they're" : 'they are',
		"theyre" : 'they are',
		"theyve" : 'they have',
		"wasn't" : 'was not',
		"we'll" : 'we will',
		"we'r" : 'we are',
		"we're" : 'we are',
		"we've" : 'we have',
		"weren't" : 'were not',
		"what's" : 'what is',
		"who'll" : 'who will',
		"who're" : 'who are',
		"who's" : 'who is',
		"why're" : 'why are',
		"won't" : 'will not',
		"would've" : 'would have',
		"wouldn't" : 'would not',
		"wouldnt" : 'would not',
		"you'll" : 'you will',
		"you're" : 'you are',
		"you've" : 'you have'
	}
	norm_tokens = []
	for t in tokens:
		if t in token_map.keys():
			for item in token_map[t].split():
				norm_tokens.append(item)
		else:
			for item in re.split('\W+', t.replace("'s","")):
				if item != '':
					norm_tokens.append(item)
	return(norm_tokens)

def get_wordnet_pos(treebank_tag):
	if treebank_tag.startswith('J'):
		return wordnet.ADJ
	elif treebank_tag.startswith('V'):
		return wordnet.VERB
	elif treebank_tag.startswith('N'):
		return wordnet.NOUN
	elif treebank_tag.startswith('R'):
		return wordnet.ADV
	else:
		return None
    
def lemma_tokenizer(text):
	text = text.replace('’',"'")
	tokens = normalize_contractions(text)


outlet           object
outlet_url       object
datetime         object
url_orig         object
headline         object
description      object
author           object
domain           object
topic_tags       object
text             object
section          object
news_keywords    object
subsection       object
paywall          object
provider         object
ideology         object
dtype: object

In [93]:
train_df['headline'] = train_df['headline'].apply(lemma_tokenizer)

In [94]:
train_df

,outlet,outlet_url,datetime,url_orig,headline,description,author,domain,topic_tags,text,section,news_keywords,subsection,paywall,provider,ideology
0,thehill,https://www.thehill.com/,2018-05-03T19:09:42Z,http://thehill.com/homenews/administration/386...,None,“i am waiting for the attorney general to step...,Niall Stanage,Administration,"Russia Investigation, Rudy Giuliani, Michael C...",rudy giuliani called for attorney general jeff...,Homenews,NaN,NaN,NaN,NaN,center
1,cnbc,https://www.cnbc.com/,2019-01-31T09:12:05Z,https://www.cnbc.com/2019/01/31/us-bonds-feder...,None,u.s. government debt prices were higher thursd...,Sam Meredith,Bonds,"Bonds, Central banking, World Markets, Bitcoin...",u.s. government debt prices were higher thursd...,NaN,"US treasury notes, bonds, economic data, gover...",NaN,NaN,NaN,center
2,thehill,https://www.thehill.com/,2019-03-23T00:17:15Z,https://thehill.com/homenews/news/435392-sam-a...,None,samuel adams is releasing a new beer inspired ...,Brooke Seipel,News,NaN,samuel adams is releasing a new beer inspired ...,Homenews,NaN,NaN,NaN,NaN,center
3,bbcnews,https://www.bbc.co.uk/news,2019-04-17T17:00:27Z,https://www.bbc.co.uk/news/health-47960874,None,the study could aid medical research and fuel ...,James Gallagher,Health,"Alzheimer's, Pigs, United States, Medicine, Me...",us scientists have partially revived pig brain...,News,NaN,NaN,NaN,NaN,center
4,cnbc,https://www.cnbc.com/,2019-02-25T08:33:42Z,https://www.cnbc.com/2019/02/25/stock-market-i...,None,u.s. stock index futures pointed to a higher s...,Spriha Srivastava,U.S. Markets,"Xi Jinping, Breaking News: Markets, Pre-market...",u.s. stock index futures pointed to a higher s...,NaN,"US stock futures, Wall Street pre-markets, US ...",NaN,NaN,NaN,center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,breitbart,https://www.breitbart.com/,2018-08-16T19:29:14Z,https://www.breitbart.com/big-government/2018/...,None,president donald trump opened a cabinet meetin...,Charlie Spiering,Big Government,"Cabinet Meeting, Donald Trump, Freedom Of The ...","“if you’d like, you can stay. if you’d like, y...",NaN,NaN,NaN,NaN,NaN,right
9996,infowars,https://www.infowars.com/,2018-05-04T09:39:42Z,https://www.infowars.com/mueller-goes-after-ro...,None,focus on alleged interactions between former t...,Brian Schwartz,Featured Stories,NaN,special counsel robert mueller is focusing int...,NaN,NaN,NaN,NaN,NaN,right
9997,breitbart,https://www.breitbart.com/,2019-04-11T21:26:12Z,https://www.breitbart.com/national-security/20...,None,julian assange's arrest left many questioning ...,Frances Martel,National Security,"Bradley Manning, Ecuador, Julian Assange, Lati...",the move by president lenin moreno – a sociali...,NaN,NaN,NaN,NaN,NaN,right
9998,breitbart,https://www.breitbart.com/,2018-11-17T01:36:21Z,https://www.breitbart.com/tech/2018/11/16/flas...,None,hillary clinton's office ignored julian assang...,Allum Bokhari,Tech,"Clinton Email Scandal, Hillary Clinton, Hillar...",the department of justice has obtained sealed ...,NaN,NaN,NaN,NaN,NaN,right
